<a href="https://colab.research.google.com/github/mmhs29/PDS-12-30-Task-1/blob/main/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import os
from datetime import datetime
from threading import Lock

# File paths
inventory_file = '/content/3_Inventory_Management_System.csv'
sales_file = 'sales_records.csv'
file_lock = Lock()

# Helper function to read CSV
def read_csv(file_path):
    try:
        with open(file_path, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            return list(reader)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}.")
        return []
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

# Helper function to write to CSV
def write_csv(file_path, data, fieldnames):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Task 1: Add a new item to the inventory
def add_item():
    inventory = read_csv(inventory_file)
    if not inventory:
        print("Error: Unable to load inventory.")
        return

    fieldnames = inventory[0].keys() if inventory else ["Item ID", "Item Name", "Category", "Quantity", "Unit Price", "Reorder Level"]

    item_id = input("Enter Item ID: ").strip()
    item_name = input("Enter Item Name: ")
    category = input("Enter Category: ")
    quantity = int(input("Enter Quantity: "))
    unit_price = float(input("Enter Unit Price: "))
    reorder_level = int(input("Enter Reorder Level: "))

    inventory.append({
        "Item ID": item_id,
        "Item Name": item_name,
        "Category": category,
        "Quantity": quantity,
        "Unit Price": unit_price,
        "Reorder Level": reorder_level
    })

    write_csv(inventory_file, inventory, fieldnames)
    print("Item added successfully!")

# Task 2: Update inventory item
def update_item():
    inventory = read_csv(inventory_file)
    if not inventory:
        print("Error: Unable to load inventory.")
        return

    item_id = input("Enter the Item ID to update: ").strip()

    for item in inventory:
        if item["Item ID"].strip().lower() == item_id.lower():
            print(f"Current details: {item}")
            item["Item Name"] = input(f"Enter new name (or press Enter to keep '{item['Item Name']}'): ") or item["Item Name"]
            item["Category"] = input(f"Enter new category (or press Enter to keep '{item['Category']}'): ") or item["Category"]
            item["Quantity"] = int(input(f"Enter new quantity (or press Enter to keep '{item['Quantity']}'): ") or item["Quantity"])
            item["Unit Price"] = float(input(f"Enter new unit price (or press Enter to keep '{item['Unit Price']}'): ") or item["Unit Price"])
            item["Reorder Level"] = int(input(f"Enter new reorder level (or press Enter to keep '{item['Reorder Level']}'): ") or item["Reorder Level"])
            write_csv(inventory_file, inventory, inventory[0].keys())
            print("Item updated successfully!")
            return
    print("Item not found!")

# Task 3: Delete an inventory item
def delete_item():
    inventory = read_csv(inventory_file)
    if not inventory:
        print("Error: Unable to load inventory.")
        return

    item_id = input("Enter the Item ID to delete: ").strip()

    updated_inventory = [item for item in inventory if item["Item ID"].strip().lower() != item_id.lower()]
    if len(updated_inventory) < len(inventory):
        write_csv(inventory_file, updated_inventory, inventory[0].keys())
        print("Item deleted successfully!")
    else:
        print("Item not found!")

# Task 4: Monitor low stock levels
def monitor_low_stock():
    inventory = read_csv(inventory_file)
    if not inventory:
        print("Error: Unable to load inventory.")
        return

    low_stock_items = [item for item in inventory if int(item["Quantity"]) <= int(item["Reorder Level"])]

    print("\nLow Stock Items:")
    for item in low_stock_items:
        print(item)

# Task 5: Record a sale
def record_sale():
    inventory = read_csv(inventory_file)
    if not inventory:
        print("Error: Unable to load inventory.")
        return

    fieldnames = inventory[0].keys()

    item_id = input("Enter Item ID: ").strip()
    try:
        quantity_sold = int(input("Enter quantity sold: "))
        if quantity_sold <= 0:
            print("Error: Quantity sold must be a positive number.")
            return
    except ValueError:
        print("Error: Please enter a valid number for quantity sold.")
        return

    for item in inventory:
        if item["Item ID"].strip().lower() == item_id.lower():
            if int(item["Quantity"]) >= quantity_sold:
                item["Quantity"] = str(int(item["Quantity"]) - quantity_sold)

                # Save sale record
                with file_lock:
                    if not os.path.exists(sales_file):
                        with open(sales_file, mode='w', newline='', encoding='utf-8') as file:
                            writer = csv.writer(file)
                            writer.writerow(["Date", "Item ID", "Item Name", "Quantity Sold", "Total Price"])

                    with open(sales_file, mode='a', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        total_price = quantity_sold * float(item["Unit Price"])
                        writer.writerow([datetime.now().strftime('%Y-%m-%d %H:%M:%S'), item_id, item["Item Name"], quantity_sold, total_price])

                write_csv(inventory_file, inventory, fieldnames)
                print(f"Sale recorded! Total price: ${total_price:.2f}")
                return
            else:
                print("Insufficient stock!")
                return
    print("Item not found!")

# Task 6: Main menu
def main_menu():
    while True:
        print("\nInventory Management System")
        print("1. Add Item")
        print("2. Update Item")
        print("3. Delete Item")
        print("4. Monitor Low Stock")
        print("5. Record Sale")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            add_item()
        elif choice == '2':
            update_item()
        elif choice == '3':
            delete_item()
        elif choice == '4':
            monitor_low_stock()
        elif choice == '5':
            record_sale()
        elif choice == '6':
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main_menu()


Inventory Management System
1. Add Item
2. Update Item
3. Delete Item
4. Monitor Low Stock
5. Record Sale
6. Exit
